A simpel sine wave is an IMF. An IMF have a zero mean.

*   List item
*   List item



In [ ]:
import sys
import scipy.io
import numpy as np
import matplotlib.pyplot as plt

#setting
minus_step = 9
frequency = 1
total_time = 1
res = 0.01

time = np.arange(0, total_time, res)
amplitude = np.sin(2 * np.pi * frequency * total_time 
                   * (time/time[-1]))
np.mean(amplitude)

The difference between minima and maxima of IMF is at most by 1.

In [ ]:
nodata_flag = np.nan
envelope_max = np.zeros(len(amplitude))
envelope_min = np.zeros(len(amplitude))
envelope_max[:] = nodata_flag
envelope_min[:] = nodata_flag

minima_total = 0
maxima_total = 0
graph_stat_history = 'up'
graph_stat_recent = 'up'
for i in range(len(amplitude)):
  if i == 1:
    if amplitude[i] > amplitude[i-1]:
      graph_stat_recent = 'up'
      graph_stat_history = 'up'
    elif amplitude[i] < amplitude[i-1]:
      graph_stat_recent = 'down'
      graph_stat_history = 'down'
    else:
      graph_stat_recent = graph_stat_history
    continue
  if amplitude[i] > amplitude[i-1]:
    graph_stat_recent = 'up'
  elif amplitude[i] < amplitude[i-1]:
    graph_stat_recent = 'down'
  else:
    graph_stat_recent = graph_stat_history
	
  if graph_stat_recent != graph_stat_history:
    if graph_stat_recent == 'up':
      envelope_min[i] = amplitude[i-1]
      minima_total += 1
      graph_stat_history = 'up'
    elif graph_stat_recent == 'down':
      envelope_max[i] = amplitude[i-1]
      maxima_total += 1
      graph_stat_history = 'down'

print('Total minima', minima_total)
print('Total maxima', maxima_total)
plt.title('A Sine Wave (Red dot = maxima, Blue dot = minima)')
plt.plot(time, amplitude)
plt.plot(time, envelope_min, 'bo')
plt.plot(time, envelope_max, 'ro')
plt.show()

In this article, I will show you the explanation of EMD algorithm that easy to understand (using practical code and many graph plot). First, let's decide our toy test signal that will be decomposed using EMD. In this article, I will merge 1 Hz and 4 Hz sine wave. It will be like this.



In [ ]:
frequency_1 = 1
frequency_2 = 4
total_time_test = 5
res_test = 0.01

time_test = np.arange(0, total_time_test, res_test)
amplitude_1 = np.sin(2 * np.pi * frequency_1 * total_time_test \
                     * (time_test/time_test[-1]))
amplitude_2 = np.sin(2 * np.pi * frequency_2 * total_time_test \
                     * (time_test/time_test[-1]))
amplitude_fusion = amplitude_1 + amplitude_2
plt.plot(time_test, amplitude_1)
plt.title('%s Hz Wave'%(frequency_1))
plt.show()
plt.close()
plt.plot(time_test, amplitude_2)
plt.title('%s Hz Wave'%(frequency_2))
plt.show()
plt.close()
plt.plot(time_test, amplitude_fusion)
plt.title('A Fusion Wave')
plt.show()
plt.close()

**Step 1, Find the Minima and Maxima**

In [ ]:
nodata_flag = np.nan
envelope_max_test = np.zeros(len(amplitude_fusion))
envelope_min_test = np.zeros(len(amplitude_fusion))
envelope_max_test[:] = nodata_flag
envelope_min_test[:] = nodata_flag

minima_total = 0
maxima_total = 0
graph_stat_history = 'up'
graph_stat_recent = 'up'
for i in range(len(amplitude_fusion)):
  if i == 1:
    if amplitude_fusion[i] > amplitude_fusion[i-1]:
      graph_stat_recent = 'up'
      graph_stat_history = 'up'
    elif amplitude_fusion[i] < amplitude_fusion[i-1]:
      graph_stat_recent = 'down'
      graph_stat_history = 'down'
    else:
      graph_stat_recent = graph_stat_history
    continue
  if amplitude_fusion[i] > amplitude_fusion[i-1]:
    graph_stat_recent = 'up'
  elif amplitude_fusion[i] < amplitude_fusion[i-1]:
    graph_stat_recent = 'down'
  else:
    graph_stat_recent = graph_stat_history
	
  if graph_stat_recent != graph_stat_history:
    if graph_stat_recent == 'up':
      envelope_min_test[i] = amplitude_fusion[i-1]
      minima_total += 1
      graph_stat_history = 'up'
    elif graph_stat_recent == 'down':
      envelope_max_test[i] = amplitude_fusion[i-1]
      maxima_total += 1
      graph_stat_history = 'down'

print(maxima_total)
print(minima_total)
plt.title('A Fusion Wave')
plt.plot(time_test, amplitude_fusion)
plt.plot(time_test, envelope_min_test, 'bo')
plt.plot(time_test, envelope_max_test, 'ro')

**Step 2, create an envelope of minima and maxima from the array of minima and maxima**. Interpolate the value of minima and maxima to create an envelope of minima and maxima using a cubic spline.

In [ ]:
from scipy.interpolate import CubicSpline

all_minima_point = []
all_minima_point_time = []
all_maxima_point = []
all_maxima_point_time = []
for i in range(len(envelope_min_test)):
  if not np.isnan(envelope_min_test[i]):
    all_minima_point.append(envelope_min_test[i])
    all_minima_point_time.append(time_test[i])

for i in range(len(envelope_max_test)):
  if not np.isnan(envelope_max_test[i]):
    all_maxima_point.append(envelope_max_test[i])
    all_maxima_point_time.append(time_test[i])

all_minima_point = np.asarray(all_minima_point)
all_minima_point_time = np.asarray(all_minima_point_time)
all_maxima_point = np.asarray(all_maxima_point)
all_maxima_point_time = np.asarray(all_maxima_point_time)

#print(all_maxima_point, all_minima_point)
cs_minima = CubicSpline(all_minima_point_time, \
                        all_minima_point)
cs_maxima = CubicSpline(all_maxima_point_time, \
                        all_maxima_point)
#print(cs_minima, cs_maxima)
envelope_minima_cs = cs_minima(time_test)
envelope_maxima_cs = cs_maxima(time_test)
#print(envelope_minima_cs)

plt.title('A Fusion Wave')
plt.plot(time_test, amplitude_fusion)
plt.plot(time_test, envelope_min_test, 'bo')
plt.plot(time_test, envelope_max_test, 'ro')
plt.plot(time_test, envelope_minima_cs)
plt.plot(time_test, envelope_maxima_cs)

**Step 4, from the envelope of minima and maxima, get the middle value**.

In [ ]:
mid_value = np.zeros(len(envelope_maxima_cs))
for i in range(len(mid_value)):
  mid_value[i] = (envelope_maxima_cs[i] + \
                  envelope_minima_cs[i]) / 2

plt.title('A Fusion Wave')
plt.plot(time_test, amplitude_fusion)
plt.plot(time_test, envelope_min_test, 'bo')
plt.plot(time_test, envelope_max_test, 'ro')
plt.plot(time_test, envelope_minima_cs)
plt.plot(time_test, envelope_maxima_cs)
plt.plot(time_test, mid_value)

**Step 5, decrease the value of the real toy test signal by the middle value of the envelope**.

In [ ]:
extracted_val = amplitude_fusion - mid_value
plt.title('An Extracted Wave')
plt.plot(time_test, extracted_val)

If you look closely, this is our 4 Hz sine wave successfully extracted. But looks like there is some error like a sudden leap on both sides of the wave. **This has come from the error of the interpolation of the cubic spline. The interpolation of the cubic spline becomes so bad at the beginning and the end of the signal.** The solution is we must define a boundary condition. What I mean about boundary condition is **we must define the envelope of minima and maxima at the beginning and the end of the signal before doing the cubic spline**. In this article, the envelope of minima and maxima of the beginning and the end of the signal is the same as with the nearest maxima and minima from them.

In [ ]:
nodata_flag = np.nan
envelope_max_test = np.zeros(len(amplitude_fusion))
envelope_min_test = np.zeros(len(amplitude_fusion))
envelope_max_test[:] = nodata_flag
envelope_min_test[:] = nodata_flag

minima_total = 0
maxima_total = 0
graph_stat_history = 'up'
graph_stat_recent = 'up'
for i in range(len(amplitude_fusion)):
  if i == 1:
    if amplitude_fusion[i] > amplitude_fusion[i-1]:
      graph_stat_recent = 'up'
      graph_stat_history = 'up'
    elif amplitude_fusion[i] < amplitude_fusion[i-1]:
      graph_stat_recent = 'down'
      graph_stat_history = 'down'
    else:
      graph_stat_recent = graph_stat_history
    continue
  if amplitude_fusion[i] > amplitude_fusion[i-1]:
    graph_stat_recent = 'up'
  elif amplitude_fusion[i] < amplitude_fusion[i-1]:
    graph_stat_recent = 'down'
  else:
    graph_stat_recent = graph_stat_history
	
  if graph_stat_recent != graph_stat_history:
    if graph_stat_recent == 'up':
      envelope_min_test[i] = amplitude_fusion[i-1]
      minima_total += 1
      graph_stat_history = 'up'
    elif graph_stat_recent == 'down':
      envelope_max_test[i] = amplitude_fusion[i-1]
      maxima_total += 1
      graph_stat_history = 'down'

#boundary condition of minima
for i in range(len(envelope_min_test)):
  if not np.isnan(envelope_min_test[i]):
    envelope_min_test[0] = envelope_min_test[i]
    break
envelope_min_test_reverse = envelope_min_test[::-1]
for i in range(len(envelope_min_test_reverse)):
  if not np.isnan(envelope_min_test_reverse[i]):
    envelope_min_test[-1] = envelope_min_test_reverse[i]
    break

#boundary condition of maxima
for i in range(len(envelope_max_test)):
  if not np.isnan(envelope_max_test[i]):
    envelope_max_test[0] = envelope_max_test[i]
    break
envelope_max_test_reverse = envelope_max_test[::-1]
for i in range(len(envelope_max_test_reverse)):
  if not np.isnan(envelope_max_test_reverse[i]):
    envelope_max_test[-1] = envelope_max_test_reverse[i]
    break

all_minima_point = []
all_minima_point_time = []
all_maxima_point = []
all_maxima_point_time = []
for i in range(len(envelope_min_test)):
  if not np.isnan(envelope_min_test[i]):
    all_minima_point.append(envelope_min_test[i])
    all_minima_point_time.append(time_test[i])

for i in range(len(envelope_max_test)):
  if not np.isnan(envelope_max_test[i]):
    all_maxima_point.append(envelope_max_test[i])
    all_maxima_point_time.append(time_test[i])

all_minima_point = np.asarray(all_minima_point)
all_minima_point_time = np.asarray(all_minima_point_time)
all_maxima_point = np.asarray(all_maxima_point)
all_maxima_point_time = np.asarray(all_maxima_point_time)

#print(all_maxima_point, all_minima_point)
cs_minima = CubicSpline(all_minima_point_time, 
                        all_minima_point)
cs_maxima = CubicSpline(all_maxima_point_time, 
                        all_maxima_point)
#print(cs_minima, cs_maxima)
envelope_minima_cs = cs_minima(time_test)
envelope_maxima_cs = cs_maxima(time_test)
#print(envelope_minima_cs)

mid_value = np.zeros(len(envelope_maxima_cs))
for i in range(len(mid_value)):
  mid_value[i] = (envelope_maxima_cs[i] + 
                  envelope_minima_cs[i])/2

plt.title('A Fusion Wave')
plt.plot(time_test, amplitude_fusion)
plt.plot(time_test, envelope_min_test, 'bo')
plt.plot(time_test, envelope_max_test, 'ro')
plt.plot(time_test, envelope_minima_cs)
plt.plot(time_test, envelope_maxima_cs)
plt.plot(time_test, mid_value)
plt.show()
plt.close()

extracted_val = amplitude_fusion - mid_value
plt.title('An Extracted Wave')
plt.plot(time_test, extracted_val)
plt.show()
plt.close()


Ouch, this time the error is affected the entire extracted signal. But the sudden leap has vanished.

**Step 6, Check if this extracted signal is an IMF**.

In [ ]:
np.mean(extracted_val)

minima_total = 0
maxima_total = 0
graph_stat_history = 'up'
graph_stat_recent = 'up'
for i in range(len(extracted_val)):
  if i == 1:
    if extracted_val[i] > extracted_val[i-1]:
      graph_stat_recent = 'up'
      graph_stat_history = 'up'
    elif extracted_val[i] < extracted_val[i-1]:
      graph_stat_recent = 'down'
      graph_stat_history = 'down'
    else:
      graph_stat_recent = graph_stat_history
    continue
  if extracted_val[i] > extracted_val[i-1]:
    graph_stat_recent = 'up'
  elif extracted_val[i] < extracted_val[i-1]:
    graph_stat_recent = 'down'
  else:
    graph_stat_recent = graph_stat_history
	
  if graph_stat_recent != graph_stat_history:
    if graph_stat_recent == 'up':
      minima_total += 1
      graph_stat_history = 'up'
    elif graph_stat_recent == 'down':
      maxima_total += 1
      graph_stat_history = 'down'

print('mean:', np.mean(extracted_val))
print('Total minima', minima_total)
print('Total maxima', maxima_total)

The mean is nearly zero, I think we can ignore it and round it to zero. The number of Minima and Maxima also fulfill the requirement, **we get an IMF**. If you are not satisfied with the mean of the extracted signal, you can process it 1 more time following step 2 - step 6. But **this time, instead of inputting a real toy test signal, we use that extracted signal**. This the route of the algorithm when in the first or second or so on triest, the extracted signal not satisfied IMF condition. To make it more simple, let's just admit the signal is an IMF.

**Step 7, decrease the original toy test signal with this IMF.**

In [ ]:
amplitude_fusion = amplitude_fusion - extracted_val

plt.title('A Fusion Wave')
plt.plot(time_test, amplitude_fusion)
plt.show()
plt.close()


Woah, we get our 1 Hz signal (but as we thought, the beginning and the end is wrong). It looks like we success to decompose our fused signal. But, the condition to stop in the algorithm of EMD is when our residuum signal is just a constant, monotonic, or just have 1 extremum (just have 1 minima or maxima). **Our signal which we have reduced by the IMF before is called residuum**. Let's continue the process.

In [ ]:
envelope_max_test = np.zeros(len(amplitude_fusion))
envelope_min_test = np.zeros(len(amplitude_fusion))
envelope_max_test[:] = nodata_flag
envelope_min_test[:] = nodata_flag

minima_total = 0
maxima_total = 0
graph_stat_history = 'up'
graph_stat_recent = 'up'
for i in range(len(amplitude_fusion)):
  if i == 1:
    if amplitude_fusion[i] > amplitude_fusion[i-1]:
      graph_stat_recent = 'up'
      graph_stat_history = 'up'
    elif amplitude_fusion[i] < amplitude_fusion[i-1]:
      graph_stat_recent = 'down'
      graph_stat_history = 'down'
    else:
      graph_stat_recent = graph_stat_history
    continue
  if amplitude_fusion[i] > amplitude_fusion[i-1]:
    graph_stat_recent = 'up'
  elif amplitude_fusion[i] < amplitude_fusion[i-1]:
    graph_stat_recent = 'down'
  else:
    graph_stat_recent = graph_stat_history
	
  if graph_stat_recent != graph_stat_history:
    if graph_stat_recent == 'up':
      envelope_min_test[i] = amplitude_fusion[i-1]
      minima_total += 1
      graph_stat_history = 'up'
    elif graph_stat_recent == 'down':
      envelope_max_test[i] = amplitude_fusion[i-1]
      maxima_total += 1
      graph_stat_history = 'down'

all_minima_point = []
all_minima_point_time = []
all_maxima_point = []
all_maxima_point_time = []
for i in range(len(envelope_min_test)):
  if not np.isnan(envelope_min_test[i]):
    all_minima_point.append(envelope_min_test[i])
    all_minima_point_time.append(time_test[i])

for i in range(len(envelope_max_test)):
  if not np.isnan(envelope_max_test[i]):
    all_maxima_point.append(envelope_max_test[i])
    all_maxima_point_time.append(time_test[i])

all_minima_point = np.asarray(all_minima_point)
all_minima_point_time = np.asarray(all_minima_point_time)
all_maxima_point = np.asarray(all_maxima_point)
all_maxima_point_time = np.asarray(all_maxima_point_time)

#boundary condition of minima
for i in range(len(envelope_min_test)):
  if not np.isnan(envelope_min_test[i]):
    envelope_min_test[0] = envelope_min_test[i]
    break
envelope_min_test_reverse = envelope_min_test[::-1]
for i in range(len(envelope_min_test_reverse)):
  if not np.isnan(envelope_min_test_reverse[i]):
    envelope_min_test[-1] = envelope_min_test_reverse[i]
    break

#boundary condition of maxima
for i in range(len(envelope_max_test)):
  if not np.isnan(envelope_max_test[i]):
    envelope_max_test[0] = envelope_max_test[i]
    break
envelope_max_test_reverse = envelope_max_test[::-1]
for i in range(len(envelope_max_test_reverse)):
  if not np.isnan(envelope_max_test_reverse[i]):
    envelope_max_test[-1] = envelope_max_test_reverse[i]
    break

#print(all_maxima_point, all_minima_point)
cs_minima = CubicSpline(all_minima_point_time, 
                        all_minima_point)
cs_maxima = CubicSpline(all_maxima_point_time, 
                        all_maxima_point)
#print(cs_minima, cs_maxima)
envelope_minima_cs = cs_minima(time_test)
envelope_maxima_cs = cs_maxima(time_test)
#print(envelope_minima_cs)

mid_value = np.zeros(len(envelope_maxima_cs))
for i in range(len(mid_value)):
  mid_value[i] = (envelope_maxima_cs[i] + 
                  envelope_minima_cs[i]) / 2

plt.title('A Fusion Wave')
plt.plot(time_test, amplitude_fusion)
plt.plot(time_test, envelope_min_test, 'bo')
plt.plot(time_test, envelope_max_test, 'ro')
plt.plot(time_test, envelope_minima_cs)
plt.plot(time_test, envelope_maxima_cs)
plt.plot(time_test, mid_value)
plt.show()
plt.close()

extracted_val_1 = amplitude_fusion - mid_value
plt.title('A Fusion Wave')
plt.plot(time_test, extracted_val_1)
plt.show()
plt.close()

print('mean:', np.mean(extracted_val))
print('Total minima', minima_total)
print('Total maxima', maxima_total)

amplitude_fusion_1 = amplitude_fusion - extracted_val_1

plt.title('A Residuum')
plt.plot(time_test, amplitude_fusion_1)
plt.show()
plt.close()


It looks like the requirement of the residuum not yet fulfilled. Let's processed it again.

In [ ]:
envelope_max_test = np.zeros(len(amplitude_fusion_1))
envelope_min_test = np.zeros(len(amplitude_fusion_1))
envelope_max_test[:] = nodata_flag
envelope_min_test[:] = nodata_flag

minima_total = 0
maxima_total = 0
graph_stat_history = 'up'
graph_stat_recent = 'up'
for i in range(len(amplitude_fusion_1)):
  if i == 1:
    if amplitude_fusion_1[i] > amplitude_fusion_1[i-1]:
      graph_stat_recent = 'up'
      graph_stat_history = 'up'
    elif amplitude_fusion_1[i] < amplitude_fusion_1[i-1]:
      graph_stat_recent = 'down'
      graph_stat_history = 'down'
    else:
      graph_stat_recent = graph_stat_history
    continue
  if amplitude_fusion_1[i] > amplitude_fusion_1[i-1]:
    graph_stat_recent = 'up'
  elif amplitude_fusion_1[i] < amplitude_fusion_1[i-1]:
    graph_stat_recent = 'down'
  else:
    graph_stat_recent = graph_stat_history
	
  if graph_stat_recent != graph_stat_history:
    if graph_stat_recent == 'up':
      envelope_min_test[i] = amplitude_fusion_1[i-1]
      minima_total += 1
      graph_stat_history = 'up'
    elif graph_stat_recent == 'down':
      envelope_max_test[i] = amplitude_fusion_1[i-1]
      maxima_total += 1
      graph_stat_history = 'down'

all_minima_point = []
all_minima_point_time = []
all_maxima_point = []
all_maxima_point_time = []
for i in range(len(envelope_min_test)):
  if not np.isnan(envelope_min_test[i]):
    all_minima_point.append(envelope_min_test[i])
    all_minima_point_time.append(time_test[i])

for i in range(len(envelope_max_test)):
  if not np.isnan(envelope_max_test[i]):
    all_maxima_point.append(envelope_max_test[i])
    all_maxima_point_time.append(time_test[i])

all_minima_point = np.asarray(all_minima_point)
all_minima_point_time = np.asarray(all_minima_point_time)
all_maxima_point = np.asarray(all_maxima_point)
all_maxima_point_time = np.asarray(all_maxima_point_time)

#boundary condition of minima
for i in range(len(envelope_min_test)):
  if not np.isnan(envelope_min_test[i]):
    envelope_min_test[0] = envelope_min_test[i]
    break
envelope_min_test_reverse = envelope_min_test[::-1]
for i in range(len(envelope_min_test_reverse)):
  if not np.isnan(envelope_min_test_reverse[i]):
    envelope_min_test[-1] = envelope_min_test_reverse[i]
    break

#boundary condition of maxima
for i in range(len(envelope_max_test)):
  if not np.isnan(envelope_max_test[i]):
    envelope_max_test[0] = envelope_max_test[i]
    break
envelope_max_test_reverse = envelope_max_test[::-1]
for i in range(len(envelope_max_test_reverse)):
  if not np.isnan(envelope_max_test_reverse[i]):
    envelope_max_test[-1] = envelope_max_test_reverse[i]
    break

#print(all_maxima_point, all_minima_point)
cs_minima = CubicSpline(all_minima_point_time, all_minima_point)
cs_maxima = CubicSpline(all_maxima_point_time, all_maxima_point)
#print(cs_minima, cs_maxima)
envelope_minima_cs = cs_minima(time_test)
envelope_maxima_cs = cs_maxima(time_test)
#print(envelope_minima_cs)

mid_value = np.zeros(len(envelope_maxima_cs))
for i in range(len(mid_value)):
  mid_value[i] = (envelope_maxima_cs[i] + envelope_minima_cs[i]) / 2

plt.title('A Fusion Wave')
plt.plot(time_test, amplitude_fusion_1)
plt.plot(time_test, envelope_min_test, 'bo')
plt.plot(time_test, envelope_max_test, 'ro')
plt.plot(time_test, envelope_minima_cs)
plt.plot(time_test, envelope_maxima_cs)
plt.plot(time_test, mid_value)
plt.show()
plt.close()

extracted_val_final = amplitude_fusion_1 - mid_value
plt.title('A Fusion Wave')
plt.plot(time_test, extracted_val_final)
plt.show()
plt.close()

print('mean:', np.mean(extracted_val))
print('Total minima', minima_total)
print('Total maxima', maxima_total)

amplitude_fusion_final = amplitude_fusion_1 - extracted_val_final

plt.title('A Residuum')
plt.plot(time_test, amplitude_fusion_final)
plt.show()
plt.close()

And at last, we find the residuum (monotonic). The last extracted looks like just noise because of the accumulative error of boundary. **This is can be prevented by raising the threshold of the mean of the signal**. When the mean of the signal is not fulfilling the threshold (the extracted signal is not an IMF), it will try again following step 2 - 6 until the extracted signal fulfill the threshold. The more convergence the threshold to zero, the more likely this noise vanishes. So that this article is not too long, in the EMD algorithm demonstration earlier, the threshold we used was not too strict. Because in the end what we want to achieve here is that readers understand step by step the EMD algorithm easily. See you.